In [16]:
from pyscipopt import Model, quicksum
import numpy as np
from itertools import product
import json

In [17]:
#Importing constant variables
with open('./input_variables.json', 'r') as file:
    input_variables = json.load(file)

# print(input_variables["Qm"])

In [18]:
def instanSacy(n_customers, n_vehicles, max_demand, max_distance):
    No = set(np.arange(1,n_customers+1)) #Set of customers
    N = No | {0} #Customers + depot
    NP = No | {"P"}
    F = {"F1", "F2"} #Set of charging stations
    NF = No | F #Customers + charging stations
    G = N | F #Customers, depot and charging stations
    GP = No | F | {"P"} #Customers, Park and charging stations
    Gtotal = G | GP
    A = [(i,j) for i,j in product(Gtotal,Gtotal) if i!=j] #Set of arcs between the nodes
    
    demand = {i: (0 if i in (F | {0} | {"P"}) else int(np.random.randint(100, max_demand, 1)[0])) for i in Gtotal} #Demand per customer

    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    '''Time cost as a function of distance and avg. speed'''
    distance =  {(i,j):(0 if (i,j) == (0,"P") else int(np.random.randint(50, max_distance, 25)[0])) for i,j in A}
    time_cost = {(i,j):(distance[i,j] / input_variables["Avg_V"]) for i,j in A} #Travel time cost per arc
    # time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A} #Travel time cost per arc
    
    battery_capacity = input_variables["Battery_capacity_Wh"]

    '''Energy cost as a function of Energy Consumption per Km'''
    EV_avg_energy_consumption = (290) #in W font:'https://ev-database.org/cheatsheet/energy-consumption-electric-car'
    CE_avg_energy_consumption = (0.16) #Liters/km font:'https://www.smmt.co.uk/wp-content/uploads/sites/2/Heavy-CV-Fuel-Consumption-Fact-Sheet.pdf'
    EV_energy_cost = {(i,j):(EV_avg_energy_consumption*distance[i,j]) for (i,j) in A} #Energy cost per arc
    CE_energy_cost = {(i,j):(CE_avg_energy_consumption*distance[i,j]) for i,j in A} #Fuel cost per arc
    
    return No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost, GP, Gtotal, NP

In [19]:
def sacystation(No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost, GP, Gtotal,NP):
    """transp -- model for solving the transportation problem
    Parameters:
        No - set of customers
        N - customers + depot
        F - set of charging stations
        G - N + F
        A - set of arcs (i,j)
        M - set of vehicles
        energy_cost[i,j] - energy transportation cost on arc (i,j)
        demand[i] - demand of customer i
        load_capacity[m] - capacity of vehicle m
        time_cost - Travel time per arc(i,j)

    ==================================
    Problems:
    implementar tempo de recarga
    """
    
    model = Model("sacystation")

    #Binary decision variable
    x = {}
    for (i,j,m) in [(i,j,m) for i in Gtotal for j in Gtotal if i!=j for m in M]:
        x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
        
    #Load Variable
    l = {}
    for (i,j,m) in [(i,j,m) for i in Gtotal for j in Gtotal if i!=j for m in M]:
        l[i,j,m] = model.addVar(vtype="C", lb=0, ub=input_variables["Qm"], name=f"l{(i,j,m)}")
    
    #Remaining Battery Variable
    y = {}
    for (i,j,m) in [(i,j,m) for (i,j) in A for m in M]:
        y[i,j,m] = model.addVar(vtype="C", lb=0, ub=battery_capacity, name=f"y[{i},{j},{m}]")

    #Charge amount Variable
    v = {}
    for (i, m) in [(i, m) for i in F for m in M]:
        v[i, m] = model.addVar(vtype="C", lb=0, ub=battery_capacity,name=f"v[{i},{m}]")
        
    #Gambiarra // Reduce the number of cars based on the number of customers
    # Assuming that M consists of a set of homogeneous 
    # if (len(No) - len(M)) < 0:
    #     M = M[:len(No)]
    # print(len(No), len(M))

    """
    Restrições Estruturais
    """
    '''
    Routing constraints
    '''
    '''
    Each customer is visited only once
    '''        
    for j in No:
        model.addCons(quicksum(x[i,j,m] for i in G if (i,j) in A for m in M ) == 1, name=f'customer_visiting[{i},{j},{m}]')
        
    # '''
    # Each charging stations is visited at most once
    # '''
    # for i in F:
    #     model.addCons(quicksum(x[i,j,m] for j in GP if i!=j for m in M) <= 1, name=f'charging_visiting[{i},{j},{m}]')
    
    '''
    Tour connectivity
    Flow constraint
    '''
    for m in M:
        for j in NF:
            model.addCons(quicksum(x[i,j,m] for i in G if i!=j)
                        == quicksum(x[j,k,m] for k in GP if k!=j)
                        , name=f'tour_connectivity[{i},{j},{m}]')
        
    '''
    Vehicles tours
    ''' 
    for m in M:
        model.addCons(quicksum(x[0,j,m] for j in NP) == 1, name=f'vehicle_one_tour[{0},{j},{m}]')
    # for m in M:
    #     model.addCons(quicksum(x[i,"P",m] for i in G) == 1, name=f'vehicle_one_tour[{i},{"P"},{m}]')
                     
    '''
    Enforces that the vehicle starts from the depot and return to it
    after the tour
    '''    
    for m in M:
        model.addCons(quicksum(x[0,j,m] for j in GP ) ==
                      quicksum(x[i,"P",m] for i in G if i!=0), name=f'depot_to_depot[{m}]')
            
    '''
    Time limit constraint
    '''
    for m in M:
        model.addCons(quicksum(time_cost[i,j]*x[i,j,m] for (i,j) in A) <= 8, name=f'timeLimit[{m}]')
                
    # '''
    # Demand constraints
    # '''        
    # '''
    # Demand Satisfaction
    # '''
    # for m in M:
    #     for i in G:
    #         if i in (F | {0}):
    #         # if i in {0}:
    #             continue
    #         else:   
    #             model.addCons((
    #                 quicksum(l[j,i,m] for j in G if i!=j) -
    #                 quicksum(l[i,k,m] for k in G if i!=k))
    #                 == demand[i], name=f'demand_satisfaction[{i}]'
    #                 )   
        
    # '''
    # Vehicle Load Capacity
    # ''' 
    # for m in M:
    #     for (i,j) in A:
    #             model.addCons(demand[j] * x[i,j,m] <= l[i,j,m],     name=f'load_capacity[{i},{j},{m}')     
    #             model.addCons(l[i,j,m] * x[i,j,m] <= (input_variables["Qm"] - demand[i] ) , name=f'load_capacity[{i},{j},{m}')

    # """
    # Battery Constraints
    # """
    # '''
    # Initial charge at depot
    # '''
    # for m in M:
    #     for j in No:
    #         model.addCons(y[0,j,m] == input_variables["ICp"] * battery_capacity - EV_energy_cost[0,j], name=f'depot_charge[{m}]')
        
    # '''
    # Battery Satisfaction
    #  >>> Arrival's battery is > than the departing battery
    #      in the proportion of the energy cost to arrive
    # '''  
    # for i in NF:
    #     for m in M:
    #         if i!=j:
    #             model.addCons(
    #                 quicksum(y[i,j,m] for j in G if i!=j) - 
    #                 quicksum(y[k,i,m] for k in G if i!=k)
    #                 == (battery_capacity - EV_energy_cost[i,j])* (1-x[i,j,m])
    #         )     
                    
    # '''
    # Battery Capacity
    # '''
    # for m in M:
    #     for (i,j) in A:
    #         for k in G:
    #             if k!=i:
    #                 model.addCons(EV_energy_cost[i,j] * x[i,j,m] <= y[i,j,m],       name=f'battery_capacity[{i},{j},{m}')     
    #                 model.addCons(y[i,j,m] * x[i,j,m] <= (battery_capacity - EV_energy_cost[k,i]) , name=f'battery_capacity[{k},{i},{m}')
    
    # '''
    # Positive values for energy levels
    # '''
    # for j in G:
    #     for i in G:
    #         if i!=j:
    #             if j!=0:
    #                 for m in M:
    #                     model.addCons(y[j,i,m] <= (1 / input_variables['ICp']) * y[0,j,m], name=f'positive_energy_level[{j}]')
    
    # '''
    # Battery available energy when leaving a charging station
    # '''
    # for i in F | {0}:#MAYBE I'LL HAVE TO IMPLEMENT THE DEPOT LATER
    #     for j in G - {0}:
    #         if i!=j:
    #             for m in M:
    #                 if i in F:
    #                     model.addCons(y[j,m] <= v[i,m] + y[i,m] - EV_energy_cost[i,j]*x[i,j,m] + input_variables["Battery_capacity_Wh"]*(1-x[i,j,m]), name=f'battery_flow[{i},{j}]')
    #                 else:  # If i is the depot
    #                     model.addCons(y[j, m] <= y[i, m] - EV_energy_cost[i, j] * x[i, j, m] + input_variables["Battery_capacity_Wh"] * (1 - x[i, j, m]), name=f'battery_flow_depot[{i},{j},{m}]')      

    # '''
    # Maximum recharge at charging stations
    # '''
    # for i in F:
    #     for m in M:
    #         model.addCons(v[i,m] + y[i,m] >= 0, name=f'max_recharge[{i},{m}]')
    #         model.addCons(v[i,m] + y[i,m] <= 0.8 * input_variables["Battery_capacity_Wh"], name=f'max_recharge[{i},{m}]')
            
        
    
    
    # Objective - ONLY TRAVEL TIME and Station Charging
    travel_time_cost = quicksum(input_variables["Ctt"]*time_cost[i,j]*x[i,j,m] for (i,j) in A for m in M)
    station_charging_cost = quicksum(input_variables['HPstation_charge_cost'] * v[i,m] for i in F for m in M)
    
    model.setObjective(travel_time_cost, "minimize")

    model.optimize()

    model.data = x
    
    return model, station_charging_cost, y, l


In [20]:
if __name__ == "__main__":
    No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost, GP, Gtotal, NP = instanSacy(2,4,1000,100)
    
    model, station_charging_cost, y, l = sacystation(No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost, GP, Gtotal, NP) 
       
    model.optimize()
    
    print(M,No, sep='\n')
    
    EPS = 1.e-6  # a small tolerance
    x = model.data  # model.data contains the decision variables

    # Time cost
    # cost = sum(time_cost[i,j]*x[i,j,m] for i,j in A if i!=j)
    # values = cost.terms.values()
    # print("Total time:", sum(values))
    # print(f'Travel time ={(travel_time_cost.terms.values())}')
    # print(f'{time_cost=}')
    # print(f'Total time cost={sum(time_cost.values())}')
    # print(f'{load_capacity=}')
    # print(f'{demand=}')
    
    if model.getStatus() == "optimal":
        print("Optimal value:", model.getObjVal())
    else:
        print("Problem could not be solved to optimality")      
    
    sol = {}
    for (i, j, m) in x:
        if model.getVal(x[i, j, m]) > 0.5:
            sol.update({f"x({i}, {j}, {m})":model.getVal(x[i, j, m])})
    #     # else:
    #     #     print(f"x[{i}, {j}, {m}] = 0")
    


[1, 2, 3, 4]
{1, 2}
Problem could not be solved to optimality


Warning: SCIPgetSolVal can only be called in stage SOLVING with a valid solution (current stage: 10)

In [ ]:
#DEBUGGING
# total_distance = sum(distance[i, j] for i, j, m in [(i, j, m) for i, j in A for m in M] if model.getVal(x[i, j, m]) > 0.5)
# print("Total distance traveled:", total_distance)

total_energy_cost = sum(EV_energy_cost[i, j] for i, j, m in [(i, j, m) for i, j in A for m in M] if model.getVal(x[i, j, m]) > 0.5)

demandi = {}
lijm = {}
EV_costs = {}
yijm = {}
for i, j, m in [(i, j, m) for i, j in A for m in M]: 
    if model.getVal(x[i, j, m]) > 0.5:
        demandi.update({str(i):demand[i]})
        lijm.update({str((i,j,m)):model.getVal(l[i,j,m])})
        EV_costs.update({str((i,j)):EV_energy_cost[i,j]})
        yijm.update({str((i,j,m)):model.getVal(y[i,j,m])})

debug = {
    "Sols":sol,
    "demandi": demandi,
    "lijm":lijm,
    #  "total_energy_cost": total_energy_cost,
    "energy_costs": EV_costs,
    "yijm": yijm
}
file_path = "./debug.json"
with open(file_path, "w") as json_file:
    json.dump(debug, json_file, indent=4)
print(f"Input variables have been written to {file_path}")

    

Input variables have been written to ./debug.json


In [ ]:
model.writeProblem(r"D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp")
model.freeProb()

wrote problem to file D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp
